# 安装MXNET

由于公司需要，近期需要快速精通mxnet，接下来的几个星期会陆续更新关于mxnet的笔记，提供参考和备忘。第一篇介绍mxnet的安装，mxnet的安装过程十分蛋疼，个人也是摸索了许久才安装成功，期间也是遇到了各种奇奇怪怪的坑，为了避免新人少走弯路，遂将经验总结于此。

## windows上的安装
本人机器配置为Win10 + Cuda 7.5， 后续的安装以此为准。  
1.mxnet需要VS2013支持C++ 11特性  
在[Visual C++ Compiler Nov 2013 CTP](https://www.microsoft.com/en-us/download/details.aspx?id=41151)下载C++ 11版本的编译器，接着将`C:\Program Files (x86)\Microsoft Visual C++ Compiler Nov 2013 CTP`下所有同名目录中的文件覆盖到`C:\Program Files (x86)\Microsoft Visual Studio 12.0\VC`下所有同名目录下对应的文件（覆盖前记得备份）
![](http://7xikew.com1.z0.glb.clouddn.com/QQ%E6%88%AA%E5%9B%BE20161026092456.png)

2.从github克隆源码  
`git clone --recursive https://github.com/dmlc/mxnet`
这里提醒注意一定不要忘记--recursive参数，因为mxnet依赖于[DMLC通用工具包](http://dmlc.ml/)，--recursive参数可以自动加载mshadow等依赖。  

3.用Cmake生成项目工程文件，并编译项目  
打开cmake，`Where is the source code`栏里打开刚才下好的mxnet源代码目录，`Where to build the binaries`栏里指定生成工程文件和编译结果的路径，这里我填的是`C:/mxnet/build`，如图所示：
![](http://7xikew.com1.z0.glb.clouddn.com/QQ%E6%88%AA%E5%9B%BE20161026113902.png)
接着点击configure，生成配置。
![](http://7xikew.com1.z0.glb.clouddn.com/QQ%E6%88%AA%E5%9B%BE20161026100929.png)
然后我们点击generate，生成`.sln`项目文件
![](http://7xikew.com1.z0.glb.clouddn.com/QQ%E6%88%AA%E5%9B%BE20161026101950.png)
找到生成的工程文件`mxnet.sln`，用vs2013打开
![](http://7xikew.com1.z0.glb.clouddn.com/QQ%E6%88%AA%E5%9B%BE20161026102039.png)
最后，我们在项目mxnet上点击右键->生成，开始编译。
![](http://7xikew.com1.z0.glb.clouddn.com/QQ%E6%88%AA%E5%9B%BE20161026114239.png)
经过漫长的等待后，mxnet终于编译完成。
![](http://7xikew.com1.z0.glb.clouddn.com/QQ%E6%88%AA%E5%9B%BE20161026153134.png)
编译完成后，在`C:\mxnet\build\Release`目录下会生成三个文件：libmxnet.dll，libmxnet.exp，libmxnet.lib。

4.安装mxnet的python接口
接下来我们到mxnet的源代码目录：`G:\OpenSource\mxnet\python`，运行
```
python setup.py install
```
来安装mxnet的python包。
![](http://7xikew.com1.z0.glb.clouddn.com/QQ%E6%88%AA%E5%9B%BE20161026153603.png)
我们将libmxnet.dll
接着，导入mxnet的时候发生了如下的错误：
![](http://7xikew.com1.z0.glb.clouddn.com/QQ%E6%88%AA%E5%9B%BE20161026171027.png)

通过调试发现问题出在打开`libmxnet.dll`的时候，问题应该出在没有导入依赖的dll文件，但蛋疼的是我也不知道它到底依赖哪一些dll文件。

5.安装依赖  
通过一番搜索，我找到一个名为dependency walker的软件，用它打开`libmxnet.dll`，我们看到还缺少的dll文件有哪些（图中的问号）
![](http://7xikew.com1.z0.glb.clouddn.com/QQ%E6%88%AA%E5%9B%BE20161026161615.png)
这些均能dll在mxnet的release tab下找到，下载完成后将其解压到mxnet的pthon安装目录`C:\Anaconda2\Lib\site-packages\mxnet-0.7.0-py2.7.egg\mxnet`下。将这些文件放入目录后，我们测试一下能不能导入

In [1]:
import ctypes
ctypes._dlopen(r"C:\Anaconda2\Lib\site-packages\mxnet-0.7.0-py2.7.egg\mxnet\cudart64_75.dll")
ctypes._dlopen(r"C:\Anaconda2\Lib\site-packages\mxnet-0.7.0-py2.7.egg\mxnet\cublas64_75.dll")
ctypes._dlopen(r"cudnn64_5.dll")
ctypes._dlopen(r"C:\Anaconda2\Lib\site-packages\mxnet-0.7.0-py2.7.egg\mxnet\libopenblas.dll")
ctypes._dlopen(r"C:\Anaconda2\Lib\site-packages\mxnet-0.7.0-py2.7.egg\mxnet\opencv_world300.dll")
ctypes._dlopen(r"C:\Anaconda2\Lib\site-packages\mxnet-0.7.0-py2.7.egg\mxnet\opencv_core2413.dll")
ctypes._dlopen(r"vcomp120.dll")
ctypes._dlopen(r"kernel32.dll")
import mxnet as mx
print "mxnet version is:%s"%mx.__version__

mxnet version is:0.7.0


上面的代码中，我们需要手动的载入mxnet依赖的动态链接库才能导入，目前还不清楚为什么它不会自动载入，这个问题留待以后解决，目前可以先把上段代码加入到mxnet的初始化代码中。接着我们跑一跑examples/image-classification/train_mnist这个例子

In [2]:
import ctypes
ctypes._dlopen(r"C:\Anaconda2\Lib\site-packages\mxnet-0.7.0-py2.7.egg\mxnet\cudart64_75.dll")
ctypes._dlopen(r"C:\Anaconda2\Lib\site-packages\mxnet-0.7.0-py2.7.egg\mxnet\cublas64_75.dll")
ctypes._dlopen(r"cudnn64_5.dll")
ctypes._dlopen(r"C:\Anaconda2\Lib\site-packages\mxnet-0.7.0-py2.7.egg\mxnet\libopenblas.dll")
ctypes._dlopen(r"C:\Anaconda2\Lib\site-packages\mxnet-0.7.0-py2.7.egg\mxnet\opencv_world300.dll")
ctypes._dlopen(r"C:\Anaconda2\Lib\site-packages\mxnet-0.7.0-py2.7.egg\mxnet\opencv_core2413.dll")
ctypes._dlopen(r"vcomp120.dll")
ctypes._dlopen(r"kernel32.dll")
import mxnet as mx
import argparse
import os, sys
import logging

def _download(data_dir):
    if not os.path.isdir(data_dir):
        os.system("mkdir " + data_dir)
    os.chdir(data_dir)
    if (not os.path.exists('train-images-idx3-ubyte')) or \
       (not os.path.exists('train-labels-idx1-ubyte')) or \
       (not os.path.exists('t10k-images-idx3-ubyte')) or \
       (not os.path.exists('t10k-labels-idx1-ubyte')):
        os.system("wget http://data.dmlc.ml/mxnet/data/mnist.zip")
        os.system("unzip -u mnist.zip; rm mnist.zip")
    os.chdir("..")

def get_loc(data, attr={'lr_mult':'0.01'}):
    """
    the localisation network in lenet-stn, it will increase acc about more than 1%,
    when num-epoch >=15
    """
    loc = mx.symbol.Convolution(data=data, num_filter=30, kernel=(5, 5), stride=(2,2))
    loc = mx.symbol.Activation(data = loc, act_type='relu')
    loc = mx.symbol.Pooling(data=loc, kernel=(2, 2), stride=(2, 2), pool_type='max')
    loc = mx.symbol.Convolution(data=loc, num_filter=60, kernel=(3, 3), stride=(1,1), pad=(1, 1))
    loc = mx.symbol.Activation(data = loc, act_type='relu')
    loc = mx.symbol.Pooling(data=loc, global_pool=True, kernel=(2, 2), pool_type='avg')
    loc = mx.symbol.Flatten(data=loc)
    loc = mx.symbol.FullyConnected(data=loc, num_hidden=6, name="stn_loc", attr=attr)
    return loc

def get_mlp():
    """
    multi-layer perceptron
    """
    data = mx.symbol.Variable('data')
    fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=128)
    act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
    fc2  = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 64)
    act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")
    fc3  = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=10)
    mlp  = mx.symbol.SoftmaxOutput(data = fc3, name = 'softmax')
    return mlp

def get_lenet(add_stn=False):
    """
    LeCun, Yann, Leon Bottou, Yoshua Bengio, and Patrick
    Haffner. "Gradient-based learning applied to document recognition."
    Proceedings of the IEEE (1998)
    """
    data = mx.symbol.Variable('data')
    if(add_stn):
        data = mx.sym.SpatialTransformer(data=data, loc=get_loc(data), target_shape = (28,28),
                                         transform_type="affine", sampler_type="bilinear")
    # first conv
    conv1 = mx.symbol.Convolution(data=data, kernel=(5,5), num_filter=20)
    tanh1 = mx.symbol.Activation(data=conv1, act_type="tanh")
    pool1 = mx.symbol.Pooling(data=tanh1, pool_type="max",
                              kernel=(2,2), stride=(2,2))
    # second conv
    conv2 = mx.symbol.Convolution(data=pool1, kernel=(5,5), num_filter=50)
    tanh2 = mx.symbol.Activation(data=conv2, act_type="tanh")
    pool2 = mx.symbol.Pooling(data=tanh2, pool_type="max",
                              kernel=(2,2), stride=(2,2))
    # first fullc
    flatten = mx.symbol.Flatten(data=pool2)
    fc1 = mx.symbol.FullyConnected(data=flatten, num_hidden=500)
    tanh3 = mx.symbol.Activation(data=fc1, act_type="tanh")
    # second fullc
    fc2 = mx.symbol.FullyConnected(data=tanh3, num_hidden=10)
    # loss
    lenet = mx.symbol.SoftmaxOutput(data=fc2, name='softmax')
    return lenet

def get_iterator(data_shape):
    def get_iterator_impl(args, kv):
        data_dir = args.data_dir
        """
        if '://' not in args.data_dir:
            _download(args.data_dir)
        """
        flat = False if len(data_shape) == 3 else True

        train           = mx.io.MNISTIter(
            image       = data_dir + "train-images-idx3-ubyte",
            label       = data_dir + "train-labels-idx1-ubyte",
            input_shape = data_shape,
            batch_size  = args.batch_size,
            shuffle     = True,
            flat        = flat,
            num_parts   = kv.num_workers,
            part_index  = kv.rank)

        val = mx.io.MNISTIter(
            image       = data_dir + "t10k-images-idx3-ubyte",
            label       = data_dir + "t10k-labels-idx1-ubyte",
            input_shape = data_shape,
            batch_size  = args.batch_size,
            flat        = flat,
            num_parts   = kv.num_workers,
            part_index  = kv.rank)

        return (train, val)
    return get_iterator_impl

def parse_args():
    parser = argparse.ArgumentParser(description='train an image classifer on mnist')
    parser.add_argument('--network', type=str, default='mlp',
                        choices = ['mlp', 'lenet', 'lenet-stn'],
                        help = 'the cnn to use')
    parser.add_argument('--data-dir', type=str, default='mnist/',
                        help='the input data directory')
    parser.add_argument('--gpus', type=str,
                        help='the gpus will be used, e.g "0,1,2,3"')
    parser.add_argument('--num-examples', type=int, default=60000,
                        help='the number of training examples')
    parser.add_argument('--batch-size', type=int, default=128,
                        help='the batch size')
    parser.add_argument('--lr', type=float, default=.1,
                        help='the initial learning rate')
    parser.add_argument('--model-prefix', type=str,
                        help='the prefix of the model to load/save')
    parser.add_argument('--save-model-prefix', type=str,
                        help='the prefix of the model to save')
    parser.add_argument('--num-epochs', type=int, default=10,
                        help='the number of training epochs')
    parser.add_argument('--load-epoch', type=int,
                        help="load the model on an epoch using the model-prefix")
    parser.add_argument('--kv-store', type=str, default='local',
                        help='the kvstore type')
    parser.add_argument('--lr-factor', type=float, default=1,
                        help='times the lr with a factor for every lr-factor-epoch epoch')
    parser.add_argument('--lr-factor-epoch', type=float, default=1,
                        help='the number of epoch to factor the lr, could be .5')
    return parser.parse_args(['--gpus', '0', '--data-dir', 'G:/OpenSource/mxnet/example/image-classification/mnist/'])

if __name__ == '__main__':
    args = parse_args()

    if args.network == 'mlp':
        data_shape = (784, )
        net = get_mlp()
    elif args.network == 'lenet-stn':
        data_shape = (1, 28, 28)
        net = get_lenet(True)
    else:
        data_shape = (1, 28, 28)
        net = get_lenet()

    # kvstore
    kv = mx.kvstore.create(args.kv_store)

    # logging
    head = '%(asctime)-15s Node[' + str(kv.rank) + '] %(message)s'
    
    logger = logging.getLogger()
    formatter = logging.Formatter(head)
    stdout_handler = logging.StreamHandler(sys.stdout)
    stdout_handler.setFormatter(formatter)

    logger.addHandler(stdout_handler)

    logger.setLevel(logging.INFO)
    logger.info('start with arguments %s', args)
    
    # load model
    model_prefix = args.model_prefix
    if model_prefix is not None:
        model_prefix += "-%d" % (kv.rank)
    model_args = {}
    if args.load_epoch is not None:
        assert model_prefix is not None
        tmp = mx.model.FeedForward.load(model_prefix, args.load_epoch)
        model_args = {'arg_params' : tmp.arg_params,
                      'aux_params' : tmp.aux_params,
                      'begin_epoch' : args.load_epoch}
    # save model
    save_model_prefix = args.save_model_prefix
    if save_model_prefix is None:
        save_model_prefix = model_prefix
    checkpoint = None if save_model_prefix is None else mx.callback.do_checkpoint(save_model_prefix)

    # data
    (train, val) = get_iterator(data_shape)(args, kv)

    # train
    devs = mx.cpu() if args.gpus is None else [
        mx.gpu(int(i)) for i in args.gpus.split(',')]

    epoch_size = args.num_examples / args.batch_size

    if args.kv_store == 'dist_sync':
        epoch_size /= kv.num_workers
        model_args['epoch_size'] = epoch_size

    if 'lr_factor' in args and args.lr_factor < 1:
        model_args['lr_scheduler'] = mx.lr_scheduler.FactorScheduler(
            step = max(int(epoch_size * args.lr_factor_epoch), 1),
            factor = args.lr_factor)

    if 'clip_gradient' in args and args.clip_gradient is not None:
        model_args['clip_gradient'] = args.clip_gradient

    # disable kvstore for single device
    if 'local' in kv.type and (
            args.gpus is None or len(args.gpus.split(',')) is 1):
        kv = None

    model = mx.model.FeedForward(
        ctx                = devs,
        symbol             = net,
        num_epoch          = args.num_epochs,
        learning_rate      = args.lr,
        momentum           = 0.9,
        wd                 = 0.00001,
        initializer        = mx.init.Xavier(factor_type="in", magnitude=2.34),
        **model_args)

    eval_metrics = ['accuracy']
    ## TopKAccuracy only allows top_k > 1
    for top_k in [5, 10, 20]:
        eval_metrics.append(mx.metric.create('top_k_accuracy', top_k = top_k))

    model.fit(
        X                  = train,
        eval_data          = val,
        eval_metric        = eval_metrics,
        kvstore            = kv,
        batch_end_callback = [mx.callback.Speedometer(args.batch_size, 50)],
        epoch_end_callback = checkpoint)



INFO:root:start with arguments Namespace(batch_size=128, data_dir='G:/OpenSource/mxnet/example/image-classification/mnist/', gpus='0', kv_store='local', load_epoch=None, lr=0.1, lr_factor=1, lr_factor_epoch=1, model_prefix=None, network='mlp', num_epochs=10, num_examples=60000, save_model_prefix=None)


2016-10-26 19:37:38,994 Node[0] start with arguments Namespace(batch_size=128, data_dir='G:/OpenSource/mxnet/example/image-classification/mnist/', gpus='0', kv_store='local', load_epoch=None, lr=0.1, lr_factor=1, lr_factor_epoch=1, model_prefix=None, network='mlp', num_epochs=10, num_examples=60000, save_model_prefix=None)


INFO:root:Start training with [gpu(0)]


2016-10-26 19:37:42,038 Node[0] Start training with [gpu(0)]


INFO:root:Epoch[0] Batch [50]	Speed: 23104.70 samples/sec	Train-accuracy=0.687344


...

2016-10-26 19:38:09,523 Node[0] Epoch[9] Validation-top_k_accuracy_5=0.999299


INFO:root:Epoch[9] Validation-top_k_accuracy_10=1.000000


2016-10-26 19:38:09,525 Node[0] Epoch[9] Validation-top_k_accuracy_10=1.000000


INFO:root:Epoch[9] Validation-top_k_accuracy_20=1.000000


2016-10-26 19:38:09,526 Node[0] Epoch[9] Validation-top_k_accuracy_20=1.000000


可以看到，验证集上的准确率接近于1，这说明我们的安装过程是成功的。

总结一下期间遇到的错误：  
1.
>CMake Error: The following variables are used in this project, but they are set to NOTFOUND.  
Please set them or make sure they are set and tested correctly in the CMake files:  
CUDA_cublas_LIBRARY (ADVANCED)  
    linked by target "mxnet" in directory G:/OpenSource/mxnet  
    linked by target "mxnet" in directory G:/OpenSource/mxnet  
CUDA_cublas_device_LIBRARY (ADVANCED)  
    linked by target "mxnet" in directory G:/OpenSource/mxnet  
    linked by target "mxnet" in directory G:/OpenSource/mxnet  
CUDA_curand_LIBRARY (ADVANCED)  
    linked by target "mxnet" in directory G:/OpenSource/mxnet  
    linked by target "mxnet" in directory G:/OpenSource/mxnet  
    
解决方法：
参考[github](https://github.com/dmlc/mxnet/issues/2521)上的issue，换成64位编译器。  

2.无法打开包括文件:opencv2\opencv.hpp
解决方法：  
在项目属性页的VC++标签页中的包含目录选项中加入opencv的头文件路径`G:\opencv\build\include`即可。  

3.错误	5373	error LNK2001: 无法解析的外部符号 "int __cdecl cv::_interlockedExchangeAdd(int *,int)"(?_interlockedExchangeAdd@cv@@YAHPEAHH@Z)  
解决方法：  
在项目属性页的标签页中的链接器下的附加依赖项属性中加入opencv的库文件`G:\opencv\build\x64\vc12\lib\opencv_core2413.lib`


## 参考
[mxnet配置安装](https://github.com/zhubuntu/MXNet-Learning-Note/blob/master/mxnet%E5%AE%89%E8%A3%85%E9%85%8D%E7%BD%AE.md)  
https://github.com/dmlc/mxnet/issues/655